In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from IPython.display import HTML

# Load the data
df = pd.read_csv('Fema_Data.csv')

# Extract just the date part
df['declarationDate'] = pd.to_datetime(df['declarationDate'].str[:10])
df = df.sort_values('declarationDate').reset_index(drop=True)

# Smoothed values extracted from R's loess fit
smooth_values = [36.2256,34.6277,30.3523,30.3523,30.3523,26.6703,25.8018,24.1272,24.1272,24.1272,24.1272,23.5208,23.3213,20.8464,19.9536,19.9536,19.9536,17.1637,17.1637,16.1258,16.1258,15.2877,15.2877,15.2877,15.2877,14.12,14.12,13.8751,13.6355,13.5177,12.3099,12.0085,11.5329,11.5329,11.352,11.352,11.352,11.352,11.1775,10.9289,10.8496,10.0835,10.0835,10.023,9.964,9.964,9.8501,9.6894,9.6894,9.6894,9.6894,9.6384,9.5403,9.4023,9.3587,9.3162,9.1953,9.1953,9.0488,8.9491,8.8871,8.8285,8.8285,8.8005,8.7732,8.7467,8.7209,8.6715,8.6478,8.6478,8.6023,8.6023,8.6023,8.5441,8.5441,8.5441,8.5282,8.5139,8.5139,8.5139,8.5139,8.5139,8.5013,8.4903,8.4808,8.4808,8.4728,8.4574,8.4549,8.4549,8.4538,8.4538,8.4551,8.4609,8.4609,8.4655,8.4655,8.4775,8.485,8.4933,8.5124,8.5231,8.5465,8.5861,8.615,8.615,8.615,8.615,8.615,8.6301,8.6485,8.6729,8.7389,8.9305,9.1851,9.5621,9.5621,9.5621,9.6423,9.8049,9.8868,9.8868,9.8868,9.8868,10.2892,10.5844,10.5844,10.5844,10.8346,10.8346,10.8346,10.8346,10.8346,10.8346,10.9367,10.9367,10.9367,10.9367,10.9367,10.9809,11.0202,11.0202,11.0542,11.0542,11.0542,11.0826,11.0826,11.1079,11.1327,11.181,11.2276,11.2276,11.2502,11.2724,11.3364,11.4348,11.489,11.5232,11.5232,11.5717,11.5871,11.5871,11.6588,11.6588,11.6851,11.6851,11.7336,11.7449,11.7968,11.8154,11.8154,11.8243,11.8329,11.9095,11.9095,11.915,11.9629,11.9656,11.9849,11.9768,11.9741,11.9741,11.9663,11.9075,11.8469,11.8184,11.71,11.6893,11.6464,11.4862,11.4862,11.4862,11.4862,11.4862,11.4862,11.4862,11.4862,11.463,11.463,11.463,11.463,11.463,11.463,11.463,11.463,11.3945,11.3945,11.3723,11.3505,11.3084,11.2882,11.2882,11.2321,11.2152,11.1361,11.1273,11.1201,11.1143,11.2103,11.2587,11.3699,11.3699,11.6485,11.8207,12.0439,12.0439,12.3277,12.3277,12.3277,12.3277,12.3277,12.3277,12.3277,12.3277,12.3277,12.3703,12.4134,13.2286,13.2286,13.2814,13.3348,13.61,13.6667,13.724,13.7819,13.8403,13.8403,13.8993,13.9588,14.019,14.019,14.8529,14.8529,15.1294,15.2712,15.343,15.5615,15.7841,16.011,16.011,16.011,16.011,16.011,16.011,16.011,16.8796,17.2097,17.2097,17.3777,17.6332,17.9806,17.9806,18.3357,18.6985,18.9757,19.7363,20.5282,20.5282,20.5282,20.5282,20.5282,20.5282,21.7749,22.0977,25.083,25.083,25.083,25.083,31.9351,32.9805,32.9805,32.9805]
smooth = np.array(smooth_values)

# Standard errors extracted from R's loess fit
se_values = [6.209,5.854,4.9263,4.9263,4.9263,4.1617,3.9876,3.66,3.66,3.66,3.66,3.5443,3.5067,3.0579,2.9057,2.9057,2.9057,2.4726,2.4726,2.3316,2.3316,2.2271,2.2271,2.2271,2.2271,2.0971,2.0971,2.0722,2.0487,2.0375,1.934,1.9115,1.8786,1.8786,1.8668,1.8668,1.8668,1.8668,1.8557,1.8398,1.8347,1.7846,1.7846,1.7806,1.7766,1.7766,1.769,1.758,1.758,1.758,1.758,1.7546,1.7479,1.7385,1.7355,1.7327,1.7246,1.7246,1.7151,1.7089,1.7052,1.7018,1.7018,1.7003,1.6988,1.6975,1.6962,1.6938,1.6928,1.6928,1.6909,1.6909,1.6909,1.6876,1.6876,1.6876,1.6863,1.685,1.685,1.685,1.685,1.685,1.6837,1.6823,1.681,1.681,1.6798,1.6767,1.676,1.676,1.6754,1.675,1.6749,1.6752,1.6752,1.6757,1.6757,1.6773,1.6784,1.6796,1.6827,1.6844,1.6881,1.6939,1.6975,1.6975,1.6975,1.6975,1.6975,1.6992,1.7007,1.7022,1.7053,1.7128,1.7244,1.7477,1.7477,1.7477,1.7536,1.7667,1.7739,1.7739,1.7739,1.7739,1.8146,1.8504,1.8504,1.8504,1.8853,1.8853,1.8853,1.8853,1.8853,1.8853,1.9012,1.9012,1.9012,1.9012,1.9012,1.9085,1.9152,1.9152,1.9215,1.9215,1.9215,1.927,1.927,1.9321,1.9368,1.9451,1.9521,1.9521,1.9551,1.9578,1.964,1.9692,1.9698,1.9693,1.9693,1.9677,1.9669,1.9669,1.9623,1.9623,1.9604,1.9604,1.9571,1.9564,1.955,1.9556,1.9556,1.9561,1.9569,1.9814,1.9814,1.9852,2.0548,2.0625,2.1939,2.3602,2.382,2.382,2.4344,2.5906,2.5508,2.5355,2.4815,2.4717,2.4519,2.3826,2.3826,2.3826,2.3826,2.3826,2.3826,2.3826,2.3826,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732,2.3459,2.3459,2.3373,2.3289,2.3129,2.3053,2.3053,2.2842,2.2778,2.2447,2.24,2.2355,2.2312,2.1815,2.1731,2.1553,2.1553,2.1125,2.0885,2.0607,2.0607,2.0311,2.0311,2.0311,2.0311,2.0311,2.0311,2.0311,2.0311,2.0311,2.0273,2.0235,1.977,1.977,1.9754,1.974,1.9691,1.9686,1.9682,1.9679,1.9678,1.9678,1.9679,1.9681,1.9684,1.9684,1.9864,1.9864,1.9972,2.0036,2.0071,2.0187,2.0324,2.0484,2.0484,2.0484,2.0484,2.0484,2.0484,2.0484,2.1279,2.1657,2.1657,2.1865,2.2202,2.2698,2.2698,2.3249,2.3857,2.435,2.5824,2.7532,2.7532,2.7532,2.7532,2.7532,2.7532,3.0526,3.1355,3.9814,3.9814,3.9814,3.9814,6.2423,6.6089,6.6089,6.6089]
se_array = np.array(se_values)

# Create confidence intervals using R's SE values
ci_upper = smooth + 1.96 * se_array
ci_lower = np.maximum(smooth - 1.96 * se_array, 0)  # Clamp lower bound to 0

# Create the figure
fig = go.Figure()

# Add upper confidence interval line (no fill yet)
fig.add_trace(go.Scatter(
    x=df['declarationDate'], y=ci_upper,
    mode='lines', line=dict(color='rgba(0,0,0,0)'),
    hoverinfo='skip', showlegend=False
))

# Add lower confidence interval line with fill between
fig.add_trace(go.Scatter(
    x=df['declarationDate'], y=ci_lower,
    mode='lines', line=dict(color='rgba(0,0,0,0)'),
    fill='tonexty', fillcolor='rgba(162, 59, 114, 0.1)',
    hoverinfo='skip', showlegend=False
))

# Add smooth line
fig.add_trace(go.Scatter(
    x=df['declarationDate'], y=smooth,
    mode='lines', line=dict(color='#A23B72', width=2),
    hoverinfo='skip', showlegend=False
))

# Add data points
df['declarationRequestDate_clean'] = df['declarationRequestDate'].str[:10]
hover_text = df.apply(lambda row: f"Disaster: {row['disasterName']}<br>State: {row['stateName']}<br>Request Date: {row['declarationRequestDate_clean']}<br>Days to Approval: {row['Days Diff']}", axis=1)

fig.add_trace(go.Scatter(
    x=df['declarationDate'], y=df['Days Diff'],
    mode='markers',
    marker=dict(size=8, color='#2D5016', opacity=0.6),
    text=hover_text, hoverinfo='text',
    hovertemplate='%{text}<extra></extra>',
    showlegend=False
))

# Add 2024 event markers
max_days = df['Days Diff'].max()
fig.add_trace(go.Scatter(
    x=[pd.to_datetime('2024-03-01'), pd.to_datetime('2024-06-01')],
    y=[max_days + 15, max_days + 15],
    mode='markers',
    marker=dict(size=16, color='#4169E1', opacity=0.5, symbol='diamond'),
    text=['March 2024: DHS Appropriations Act adds $20.261 billion for the Disaster Relief Fund',
          'June 2024: Congress passed $500 million flood mitigation fund'],
    hoverinfo='text', hovertemplate='%{text}<extra></extra>',
    showlegend=False
))

# Add 2025 event markers
fig.add_trace(go.Scatter(
    x=[pd.to_datetime('2025-05-01'), pd.to_datetime('2025-10-07')],
    y=[max_days + 15, max_days + 15],
    mode='markers',
    marker=dict(size=16, color='#CC0000', opacity=0.5, symbol='diamond'),
    text=['May 2025: Staff reduced by 1/3, $646M budget cut',
          'October 2025: FEMA canceled $11B in disaster payments'],
    hoverinfo='text', hovertemplate='%{text}<extra></extra>',
    showlegend=False
))

# Update layout
fig.update_layout(
    title=dict(
        text='<b>FEMA Disaster Declaration Processing Doubles in the Fourth Quarter of 2025</b><br><sub>Showing relationship between processing delays and major policy changes in 2024 and 2025</sub>',
        font=dict(size=18)
    ),
    xaxis=dict(title='Declaration Date of Disaster', zeroline=False, showline=True, showgrid=True, gridcolor='#e8e8e8'),
    yaxis=dict(title='Processing Time in Days', zeroline=False, showline=True, showgrid=True, gridcolor='#e8e8e8'),
    hovermode='closest', plot_bgcolor='white', paper_bgcolor='white',
    font=dict(family='sans-serif', size=12, color='#333333'),
    margin=dict(t=100, r=40, b=60, l=60),
    width=900, height=600,
    shapes=[
        dict(type='line', x0=pd.to_datetime('2024-03-01'), x1=pd.to_datetime('2024-03-01'), y0=0, y1=max_days, line=dict(dash='dash', color='#444444', width=1)),
        dict(type='line', x0=pd.to_datetime('2024-06-01'), x1=pd.to_datetime('2024-06-01'), y0=0, y1=max_days, line=dict(dash='dash', color='#444444', width=1)),
        dict(type='line', x0=pd.to_datetime('2025-05-01'), x1=pd.to_datetime('2025-05-01'), y0=0, y1=max_days, line=dict(dash='dash', color='#444444', width=1)),
        dict(type='line', x0=pd.to_datetime('2025-10-07'), x1=pd.to_datetime('2025-10-07'), y0=0, y1=max_days, line=dict(dash='dash', color='#444444', width=1))
    ]
)

graph_html = fig.to_html(include_plotlyjs='cdn')

# Build complete page HTML
page_html = f"""
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>FEMA Processing Speed Bump</title>
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin="">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link href="https://fonts.googleapis.com/css2?family=Lora:ital,wght@0,400;0,700;1,400;1,700&amp;display=swap" rel="stylesheet">
    <style>
        body {{
            margin: 0;
            font-size: 20px;
            font-family: Georgia, 'Times New Roman', Times, serif;
            text-rendering: optimizeLegibility;
        }}
        .content {{
            max-width: 640px;
            margin: auto;
            padding: 2em 1em;
        }}
        a {{ color: #f05349; }}
        p {{
            line-height: 1.6;
            margin: 0;
            padding-bottom: 1.2em;
        }}
        .byline {{
            font-size: 0.95em;
            color: #666;
            margin-bottom: 1.5em;
        }}
        .byline p {{
            margin: 0;
            padding: 0;
        }}
        .quote {{
            font-style: italic;
            color: #555;
            margin: 1.5em 0;
            padding-left: 1.5em;
            border-left: 4px solid #A23B72;
        }}
        .quote-attribution {{
            font-size: 0.95em;
            color: #888;
            margin: 0.5em 0 1.5em 0;
            padding: 0;
        }}
        .hero {{
            position: relative;
            width: 100%;
            height: 600px;
            overflow: hidden;
            display: flex;
            align-items: center;
            justify-content: center;
        }}
        .hero video {{
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            object-fit: cover;
        }}
        .hero-content {{
            position: absolute;
            text-align: center;
            color: #fff;
            z-index: 10;
            width: 95%;
            max-width: 1000px;
            top: 35%;
            left: 50%;
            transform: translate(-50%, -50%);
        }}
        .hero h1 {{
            color: #fff;
            font-size: 4em;
            font-weight: 900;
            margin: 0;
            padding: 1.2em 1.5em;
            letter-spacing: -1px;
            line-height: 1.1;
            display: inline-block;
            background-color: rgba(0, 0, 0, 0.45);
            border-radius: 8px;
        }}
        .graph-container {{
            width: 100%;
            display: flex;
            justify-content: center;
            margin: 3em 0 2em 0;
            padding: 0 1em;
        }}
        .graph-wrapper {{
            width: 100%;
            max-width: 900px;
        }}
        @media (max-width: 640px) {{
            body {{ font-size: 18px; }}
            .content {{ padding: 1.5em 0.5em; }}
            .hero {{ height: 400px; }}
            .hero h1 {{ font-size: 2.2em; padding: 0.8em 1em; }}
        }}
    </style>
</head>
<body>
    <div class="hero">
        <video src="FEMA.mp4" preload="auto" tabindex="0" playsinline="" autoplay="" loop="" muted=""></video>
        <div class="hero-content">
            <h1>Double Time: Tracking FEMA's Processing Speed Bump</h1>
        </div>
    </div>
    <div class="content">
        <div class="byline">
            <p>By <a href="https://dagmarroth.github.io/Website/">Dagmar Rothschild</a></p>
        </div>
        <p>Following significant budget cuts and staff reductions at FEMA in 2025, disaster declaration processing times doubled in the fourth quarter. This interactive chart shows the relationship between major policy changes and processing delays across all disaster declarations.</p>
        <div class="quote">
            "When Hurricane Helene hit our farm in western North Carolina, we lost everything—the crops, the equipment, part of the barn. We filed for disaster assistance right away, but we've been waiting months now. The delays just keep piling up, and meanwhile we're trying to figure out how to survive the winter and plant next season."
            <p class="quote-attribution">— Justin Rhodes, western North Carolina farmer</p>
        </div>
    </div>
    <div class="graph-container">
        <div class="graph-wrapper">
            {graph_html}
        </div>
    </div>
</body>
</html>
"""

HTML(page_html)